In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
#chargement des données
artists = pd.read_csv('/home/michel/Documents/développement/python/SSRS/artists.csv')
tracks = pd.read_csv('/home/michel/Documents/développement/python/SSRS/tracks.csv')

# Preprocessing générique

## Feature-engineering et filtration des données

In [3]:
#copie des données
artists_df = artists.copy()
tracks_df = tracks.copy()

In [4]:
#suppression des variables inutiles (identifiants)
tracks_df = tracks_df.drop(['id','id_artists'], axis=1)
artists_df = artists_df.drop('id', axis=1)

#suppression des fausses variables continues
tracks_df = tracks_df.drop(['mode','time_signature', 'key'], axis=1)

In [5]:
#transformation des dates de sortie en années
tracks_df['release_date'] = tracks_df['release_date'].str.extract(r'(\d{4})')
tracks_df['release_date'] = tracks_df['release_date'].astype('int64')

In [6]:
#suppression des artistes sans information de genre musical
artists_df = artists_df[artists_df['genres'] != '[]']

#suppression des morceaux antérieurs à la médiane des dates de sortie pour réduire la taille du dataset
tracks_df = tracks_df[tracks_df['release_date'] >= 1992]

In [7]:
#formatage des artistes et des genres en listes
tracks_df['artists'] = tracks_df['artists'].apply(eval)
artists_df['genres'] = artists_df['genres'].apply(eval)

In [8]:
#suppression des artistes en double
artists_df = artists_df.drop_duplicates('name', keep=False)

In [9]:
#suppression des artistes sans morceaux rattachés
import itertools

song_artists = list(set(itertools.chain.from_iterable(tracks_df['artists'])))
artists_df = artists_df[artists_df['name'].isin(song_artists)]

In [10]:
#suppression des morceaux sans artistes rattachés (2min d'exécution)

all_artists = artists_df['name'].unique().tolist()

tracks_df['artists_exist'] = tracks_df['artists'].map(lambda x: all(item in all_artists for item in x))
tracks_df = tracks_df[tracks_df['artists_exist'] == True]
tracks_df = tracks_df.drop('artists_exist', axis=1)

**Tableau des artistes** : on passe de 1 104 349 individus à 45 701 et de 5 variables à 4  
**Tableau des morceaux** : on passe de 686 672 individus à 250 555 individus et de 20 variables à 15

# Preprocessing requête-spécifique

## Feature-engineering et filtration des données

In [11]:
#fonction renvoyant la liste des genres correspondant à un artiste donné

def get_genres(artists_df, artists:list):
    
    genres = []
    for artist in artists :
        genres = list(set(genres + artists_df[artists_df['name']==artist].iloc[0,1]))
    
    return genres

In [12]:
#fonction réduisant le tableau des morceaux à ceux des artistes similaires

def similar_artists_songs(tracks_df, artists_df, artists:list):
    
    ref_genres = get_genres(artists_df, artists)

    df1 = artists_df.copy()
    df2 = tracks_df.copy()
    
    df1['similar'] = artists_df['genres'].map(lambda x: any(item in ref_genres for item in x))
    df1 = df1[df1['similar'] == True]
    
    similar_artists = df1['name'].unique().tolist() 
    
    df2['similar'] = tracks_df['artists'].map(lambda x: any(item in similar_artists for item in x))
    df2 = df2[df2['similar'] == True]
    df2 = df2.drop('similar', axis=1)
    
    return df2

In [13]:
#fonction calculant l'indice de jaccard entre deux listes de genres musicaux

def jaccard_index(ref_list, other_list):
    
    union_list = list(set(ref_list + other_list))
    inter_list = list(set(union_list) - (set(ref_list) - set(other_list)) - (set(other_list) - set(ref_list)))
    
    return len(inter_list)/len(union_list)

In [14]:
#fonction ajoutant l'indice de Jaccard pour chaque morceau par rapport à un artiste de référence

def add_jaccard(selected_tracks_df, artists_df, main_artists=list):
    
    ref_genres = get_genres(artists_df, main_artists)
    df = selected_tracks_df.copy()
    df['jaccard_index'] = selected_tracks_df['artists'].map(lambda x: jaccard_index(ref_genres, get_genres(artists_df, x)))
    
    return df

## Normalisation des variables

In [15]:
#normalisation standard (retrait de la moyenne et variance unitaire) des données quantitatives

from sklearn.preprocessing import StandardScaler

def scale(selected_tracks_df):
    
    df_qual = selected_tracks_df.select_dtypes(include='object')
    df_quant = selected_tracks_df.select_dtypes(exclude='object')
    
    scaler = StandardScaler()
    scaler.fit(df_quant)
    df_quant = pd.DataFrame(scaler.transform(df_quant), index=df_quant.index, columns=df_quant.columns)
    
    return pd.concat([df_qual, df_quant], axis=1)

## Calcul de distance

In [16]:
#fonction calculant la distance euclidienne entre deux lignes

from scipy.spatial import distance

def add_distance(scaled_selected_tracks_df, ref_track_df):
    
    return distance.cdist(scaled_selected_tracks_df.values, ref_track_df.values, 'euclidean')

## Récapitulation des étapes

In [17]:
#fonction renvoyant un dataframe contenant les morceaux les plus semblables à un morceau donné

def ssrs(tracks_df, artists_df, track_name:str, artists:list):
    
    df = similar_artists_songs(tracks_df, artists_df, artists)
    df = add_jaccard(df, artists_df, artists)
    df = scale(df)
    
    df_all = df.select_dtypes(exclude='object')
    
    #caractéristiques du morceau de référence
    df_ref = df[df['name'].apply(lambda x: x == track_name)]
    df_ref = df_ref[df_ref['artists'].apply(lambda x: x == artists)]
    df_ref = df_ref.sort_values(by='release_date').head(1) #choix du morceau le plus ancien en cas de morceaux portant le même nom
    df_ref = df_ref.select_dtypes(exclude='object')
    
    df['distance'] = add_distance(df_all, df_ref)
    df = df[df['name'].str.contains(track_name) == False] #élimination du morceau de référence dans les résultats
    df['artists'] = df['artists'].astype(str)
    df = df.sort_values(by='distance')
    df = df.groupby('artists').first() #choix du meilleur morceau pour chaque artiste
    df = df.sort_values(by='distance')
    
    return df

In [18]:
#fonction renvoyant les k meilleurs morceaux du dataframe renvoyé par la fonction précédente

def print_k_best(df, k):
    print('Les {} morceaux les plus similaires que je connais sont :\n'.format(k))
    for i in range(k):
        print("'{}' par {}\n".format(df.iloc[i,0], df.index[i]))

In [19]:
#fonction renvoyant un dataframe contenant les morceaux associés à un artiste donné

def check_artist(tracks_df, artist:str):
    return tracks_df[tracks_df['artists'].apply(lambda x: any(item == artist for item in x))]

# Tests

In [20]:
df = ssrs(tracks_df, artists_df, "Wonderwall", ['Oasis'])

In [21]:
print_k_best(df, 10)

Les 10 morceaux les plus similaires que je connais sont :

'Don't Look Back in Anger - Remastered' par ['Oasis']

'Let Me Kiss You' par ['Morrissey']

'Country House - 2012 Remaster' par ['Blur']

'The Bitter End' par ['Placebo']

'Getting Away With It (All Messed Up)' par ['James']

'Bitter Sweet Symphony' par ['The Verve']

'Just Lookin'' par ['The Charlatans']

'Beautiful Day' par ['U2']

'Plug in Baby' par ['Muse']

'Blinded By The Sun' par ['The Seahorses']



In [22]:
check_artist(tracks_df, "Guizmo")

,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
381203,Attendez-moi,57,240813,0,[Guizmo],2016,0.535,0.535,-9.997,0.327,0.3020,0.0,0.1480,0.190,97.494
462235,C'est tout,44,451632,0,[Guizmo],2012,0.597,0.825,-7.646,0.384,0.4720,0.0,0.0998,0.535,98.775
462307,T'es juste ma pote,38,197733,1,[Guizmo],2012,0.680,0.718,-4.561,0.203,0.3950,0.0,0.0776,0.389,95.730
462583,Dans 10 ans,45,235893,0,[Guizmo],2014,0.428,0.872,-4.040,0.395,0.0343,0.0,0.0797,0.439,91.880
462616,André,44,464986,0,[Guizmo],2014,0.523,0.844,-4.879,0.377,0.3340,0.0,0.6370,0.846,90.291
463109,GPG 2,45,164560,0,[Guizmo],2016,0.524,0.578,-7.419,0.193,0.3550,0.0,0.1090,0.140,132.338


In [23]:
df2 = ssrs(tracks_df, artists_df, "Attendez-moi", ['Guizmo'])

In [24]:
print_k_best(df2, 5)

Les 5 morceaux les plus similaires que je connais sont :

'GPG 2' par ['Guizmo']

'À deux pas' par ['Alpha Wann', 'Nekfeu']

'Plus Tony que Sosa' par ['PNL']

'Si j'arrêtais' par ['Vald']

'Alors Dites Pas' par ['Hugo TSR']

